In [1]:
# %% [markdown]
# ## التحليل الإحصائي المتقدم
# - اختبارات الفروض
# - تحليل التباين
# - نماذج الانحدار

# %%
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
import arabic_reshaper
from bidi.algorithm import get_display
import unicodedata 
import re 

# دالة مساعدة لمعالجة النص العربي للعرض
def get_arabic_text(text):
    if pd.isna(text): 
        return ''
    normalized_text = unicodedata.normalize('NFKC', str(text))
    reshaped_text = arabic_reshaper.reshape(normalized_text)
    bidi_text = get_display(reshaped_text)
    return bidi_text

df = None 

# تحميل البيانات
try:
    df = pd.read_csv('../data/processed/sleep_data_clean.csv', encoding='utf-8-sig')
    
    print(get_arabic_text("تم تحميل البيانات بنجاح لغرض التحليل الإحصائي!"))
    print(get_arabic_text("\n*** الأعمدة المتاحة في DataFrame بعد التحميل (قبل أي معالجة) ***"))
    print(df.columns.tolist())
    
    # ******** خطوة DEBUGGING هامة: اطبع القيم الفريدة للأعمدة الأساسية هنا ********
    if 'الجنس :' in df.columns:
        print(get_arabic_text(f"القيم الفريدة الأصلية في عمود 'الجنس :': {df['الجنس :'].unique().tolist()}"))
    if 'كم عمرك؟' in df.columns:
        print(get_arabic_text(f"القيم الفريدة الأصلية في عمود 'كم عمرك؟': {df['كم عمرك؟'].unique().tolist()}"))
    if 'هل كان نومك:' in df.columns:
        print(get_arabic_text(f"القيم الفريدة الأصلية في عمود 'هل كان نومك:': {df['هل كان نومك:'].unique().tolist()}"))
    if 'مزاج' in df.columns:
        print(get_arabic_text(f"القيم الفريدة الأصلية في عمود 'مزاج': {df['مزاج'].unique().tolist()}"))
    # تأكد من طباعة ساعات_النوم_رقمي الأصلي فقط إذا كان موجودا لتجنب الخلط
    if 'ساعات_النوم_رقمي' in df.columns:
        print(get_arabic_text(f"القيم الفريدة الأصلية في عمود 'ساعات_النوم_رقمي': {df['ساعات_النوم_رقمي'].unique().tolist()}"))
    # إذا كان هناك عمود اسمه 'ساعات_النوم' بشكل افتراضي في CSV، اطبعه أيضاً
    elif 'ساعات_النوم' in df.columns and 'ساعات_النوم_رقمي' not in df.columns:
        print(get_arabic_text(f"القيم الفريدة الأصلية في عمود 'ساعات_النوم': {df['ساعات_النوم'].unique().tolist()}"))
    
    print("------------------------------------------------------\n")

    # =========================================================================
    # خطوة حاسمة: إعادة تسمية الأعمدة وإنشاء الأعمدة المفقودة
    # =========================================================================

    # 1. إعادة تسمية الأعمدة الموجودة لتتوافق مع الأسماء المستخدمة في التحليل
    # سنقوم بإعادة تسمية 'ساعات_النوم_رقمي' إلى 'ساعات_النوم' فقط
    columns_mapping = {
        'الجنس :': 'الجنس',          
        'مزاج': 'المزاج',            
        'ساعات_النوم_رقمي': 'ساعات_النوم_Cleaned', # اسم مؤقت لتجنب التضارب
    }
    
    df.rename(columns={k: v for k, v in columns_mapping.items() if k in df.columns}, inplace=True)

    # إذا كان هناك عمود باسم 'ساعات_النوم' الأصلي ولم يتم إعادة تسميته، احذفه أو أعد تسميته
    if 'ساعات_النوم' in df.columns and 'ساعات_النوم_Cleaned' in df.columns:
        if 'ساعات_النوم' != 'ساعات_النوم_Cleaned': # لتجنب محاولة حذف عمود لا يوجد
            print(get_arabic_text("تنبيه: تم اكتشاف عمودين باسم 'ساعات_النوم'. سيتم استخدام 'ساعات_النوم_Cleaned' فقط."))
            df.drop(columns=['ساعات_النوم'], inplace=True, errors='ignore') # احذف العمود الأصلي الذي قد يسبب تضاربا
        df.rename(columns={'ساعات_النوم_Cleaned': 'ساعات_النوم'}, inplace=True) # أعد تسمية العمود النظيف إلى ساعات_النوم
    elif 'ساعات_النوم_Cleaned' in df.columns: # إذا كان العمود النظيف هو الوحيد
        df.rename(columns={'ساعات_النوم_Cleaned': 'ساعات_النوم'}, inplace=True)

    
    print(get_arabic_text("\n*** الأعمدة بعد إعادة التسمية الأولية ***"))
    print(df.columns.tolist())
    print("---------------------------------------------------\n")

    # ******** تأكد أن ساعات_النوم رقمي (الآن هذا العمود هو العمود النظيف) ********
    if 'ساعات_النوم' in df.columns:
        df['ساعات_النوم'] = pd.to_numeric(df['ساعات_النوم'], errors='coerce')
        if not df['ساعات_النوم'].isnull().all():
            df['ساعات_النوم'].fillna(df['ساعات_النوم'].mean(), inplace=True)
        else:
            df['ساعات_النوم'].fillna(0, inplace=True) 
        print(get_arabic_text("تم تحويل عمود 'ساعات_النوم' إلى رقمي بنجاح ومعالجة القيم المفقودة."))
    else:
        print(get_arabic_text("تنبيه: عمود 'ساعات_النوم' غير موجود بعد المعالجة، قد تكون مشكلة في أسماء الأعمدة الأصلية."))

    # 2. إنشاء عمود 'الفئة_العمرية' من 'كم عمرك؟'
    if 'كم عمرك؟' in df.columns:
        def clean_age_robust(age_str):
            if pd.isna(age_str):
                return np.nan
            s = str(age_str).strip().replace('\xa0', ' ').replace('\n', ' ') 
            s = s.replace('>', '').replace('<', '') 
            
            numbers = re.findall(r'\d+', s)
            
            if not numbers:
                return np.nan
            
            if len(numbers) >= 2:
                try:
                    num1 = int(numbers[0])
                    num2 = int(numbers[1])
                    return min(num1, num2)
                except ValueError:
                    return np.nan
            elif len(numbers) == 1:
                return int(numbers[0])
            else:
                return np.nan 

        df['كم عمرك؟_cleaned'] = df['كم عمرك؟'].apply(clean_age_robust)
        df['كم عمرك؟_cleaned'] = pd.to_numeric(df['كم عمرك؟_cleaned'], errors='coerce')
        
        print(get_arabic_text(f"القيم الفريدة في 'كم عمرك؟_cleaned' بعد التنظيف: {df['كم عمرك؟_cleaned'].unique().tolist()}"))
        print(get_arabic_text(f"نوع بيانات 'كم عمرك؟_cleaned': {df['كم عمرك؟_cleaned'].dtype}"))

        if df['كم عمرك؟_cleaned'].isnull().all():
            print(get_arabic_text("تنبيه: عمود 'كم عمرك؟' فارغ تمامًا أو كله NaN بعد التنظيف والتحويل الرقمي. لم يتم إنشاء 'الفئة_العمرية' بشكل مفيد."))
            df['الفئة_العمرية'] = np.nan 
        else:
            if pd.isna(df['كم عمرك؟_cleaned'].mean()):
                df['كم عمرك؟_cleaned'].fillna(25, inplace=True) 
                print(get_arabic_text("تنبيه: متوسط 'كم عمرك؟_cleaned' كان NaN. تم ملء القيم المفقودة بـ 25."))
            else:
                df['كم عمرك؟_cleaned'].fillna(df['كم عمرك؟_cleaned'].mean(), inplace=True)
            
            df['كم عمرك؟_cleaned'] = df['كم عمرك؟_cleaned'].astype(int) 
            
            if df['كم عمرك؟_cleaned'].nunique() > 1: 
                bins = [0, 18, 25, 35, 45, 60, np.inf]
                labels = [
                    get_arabic_text('أقل من 18'), 
                    get_arabic_text('18-24'), 
                    get_arabic_text('25-34'), 
                    get_arabic_text('35-44'), 
                    get_arabic_text('45-59'), 
                    get_arabic_text('60+')
                ]
                df['الفئة_العمرية'] = pd.cut(df['كم عمرك؟_cleaned'], bins=bins, labels=labels, right=False)
                print(get_arabic_text("تم إنشاء عمود 'الفئة_العمرية' بنجاح."))
                print(get_arabic_text(f"الفئات العمرية الفريدة: {df['الفئة_العمرية'].unique().tolist()}"))
            else:
                print(get_arabic_text("تنبيه: بعد التنظيف، عمود 'كم عمرك؟' يحتوي على عدد قليل جداً من القيم الفريدة لإنشاء فئات عمرية ذات معنى. لم يتم إنشاء 'الفئة_العمرية' بشكل مفيد."))
                df['الفئة_العمرية'] = np.nan 
    else:
        print(get_arabic_text("تنبيه: عمود 'كم عمرك؟' غير موجود، لم يتم إنشاء 'الفئة_العمرية'."))
        df['الفئة_العمرية'] = np.nan 

    # 3. إنشاء عمود 'جودة_النوم' من 'هل كان نومك:'
    if 'هل كان نومك:' in df.columns:
        df['هل كان نومك:'] = df['هل كان نومك:'].astype(str).str.strip()
        
        quality_map = {
            get_arabic_text('متواصل'): 5,  
            get_arabic_text('متقطع'): 1     
        }
        
        df['جودة_النوم'] = df['هل كان نومك:'].apply(get_arabic_text).map(quality_map)
        
        if not df['جودة_النوم'].empty:
            if not df['جودة_النوم'].isnull().all():
                df['جودة_النوم'] = df['جودة_النوم'].fillna(df['جودة_النوم'].mean()) 
            else:
                df['جودة_النوم'] = df['جودة_النوم'].fillna(0) 
        else:
            df['جودة_النوم'] = 0 
            
        print(get_arabic_text("تم إنشاء عمود 'جودة_النوم' بنجاح."))
        print(get_arabic_text(f"القيم الفريدة لـ 'جودة_النوم' بعد التحويل: {df['جودة_النوم'].unique().tolist()}"))

    else:
        print(get_arabic_text("تنبيه: عمود 'هل كان نومك:' غير موجود، لم يتم إنشاء 'جودة_النوم'."))
        df['جودة_النوم'] = np.nan 

    # التأكد من أن 'المزاج' رقمي
    if 'المزاج' in df.columns:
        df['المزاج'] = pd.to_numeric(df['المزاج'], errors='coerce')
        if not df['المزاج'].isnull().all():
            df['المزاج'].fillna(df['المزاج'].mean(), inplace=True)
        else:
            df['المزاج'].fillna(0, inplace=True) 
        print(get_arabic_text("تم تحويل عمود 'المزاج' إلى رقمي بنجاح ومعالجة القيم المفقودة."))
    else:
        print(get_arabic_text("تنبيه: عمود 'المزاج' غير موجود."))


    print(get_arabic_text("\n*** الأعمدة المتاحة في DataFrame بعد المعالجة الكاملة (لغرض التحليل) ***"))
    print(df.columns.tolist())
    print("---------------------------------------------------\n")

except FileNotFoundError:
    print(get_arabic_text("خطأ: ملف 'sleep_data_clean.csv' لم يتم العثور عليه. يرجى التحقق من المسار."))
except Exception as e:
    print(get_arabic_text(f"حدث خطأ أثناء تحميل أو معالجة البيانات للتحليل الإحصائي: {str(e)}"))
    
# =========================================================================
# قسم التحليلات الإحصائية - يتم تشغيله فقط إذا تم تحميل df ومعالجته بنجاح
# =========================================================================
if df is not None and not df.empty:
    # 1. اختبار T للجنسين
    if 'الجنس' in df.columns and 'ساعات_النوم' in df.columns:
        df['الجنس'] = df['الجنس'].astype(str).str.strip()
        df['الجنس'] = df['الجنس'].apply(get_arabic_text)

        print(get_arabic_text(f"القيم الفريدة في عمود 'الجنس' بعد التطهير والتطبيع: {df['الجنس'].unique().tolist()}"))

        male_val_processed = get_arabic_text('رجل')   
        female_val_processed = get_arabic_text('امرأة') 

        if male_val_processed not in df['الجنس'].unique():
            print(get_arabic_text(f"تنبيه: قيمة '{male_val_processed}' ليست موجودة في عمود 'الجنس'."))
        if female_val_processed not in df['الجنس'].unique():
            print(get_arabic_text(f"تنبيه: قيمة '{female_val_processed}' ليست موجودة في عمود 'الجنس'."))

        # هنا التأكد من أننا نختار العمود الصحيح للحصول على Series
        # بما أن المشكلة تكمن في تكرار اسم العمود 'ساعات_النوم'، فإن الطريقة الآمنة هي تحديد العمود كقائمة
        # وهذا يجبر Pandas على إرجاع DataFrame حتى لو كان عمودًا واحدًا، ثم نختار منه السلسلة بشكل صريح
        male_data_frame = df[df['الجنس'] == male_val_processed][['ساعات_النوم']].dropna()
        female_data_frame = df[df['الجنس'] == female_val_processed][['ساعات_النوم']].dropna()

        # الآن نقوم باختيار العمود لجعله Series
        male_series = male_data_frame['ساعات_النوم'] if not male_data_frame.empty else pd.Series(dtype=float)
        female_series = female_data_frame['ساعات_النوم'] if not female_data_frame.empty else pd.Series(dtype=float)
        
        # ******** DEBUGGING: فحص أنواع البيانات وقيم السلاسل قبل اختبار T ********
        print(get_arabic_text(f"\nمعلومات عن بيانات الذكور قبل اختبار T:"))
        if isinstance(male_series, pd.Series):
            print(f"  نوع البيانات: {male_series.dtype}")
            print(f"  أول 5 قيم: {male_series.head().tolist()}")
        else:
            print(get_arabic_text("  تنبيه: male_series ليس Series بشكل غير متوقع!"))
            print(f"  نوع البيانات: {type(male_series)}")
            
        print(get_arabic_text(f"معلومات عن بيانات الإناث قبل اختبار T:"))
        if isinstance(female_series, pd.Series):
            print(f"  نوع البيانات: {female_series.dtype}")
            print(f"  أول 5 قيم: {female_series.head().tolist()}")
        else:
            print(get_arabic_text("  تنبيه: female_series ليس Series بشكل غير متوقع!"))
            print(f"  نوع البيانات: {type(female_series)}")
            
        print("-" * 50)

        if not male_series.empty and not female_series.empty:
            t_result = stats.ttest_ind(male_series, female_series, nan_policy='omit') 
            print(get_arabic_text(f"نتيجة اختبار T (ساعات_النوم بين الجنسين): الإحصاء = {t_result.statistic:.2f}, القيمة-P = {t_result.pvalue:.3f}"))
        else:
            print(get_arabic_text("لا توجد بيانات كافية (ذكور أو إناث) بعد إزالة القيم المفقودة لإجراء اختبار T على ساعات النوم."))
    else:
        print(get_arabic_text("الأعمدة 'الجنس' أو 'ساعات_النوم' غير موجودة لإجراء اختبار T."))


    # 2. تحليل التباين (ANOVA)
    if 'المزاج' in df.columns and 'الفئة_العمرية' in df.columns:
        df_anova = df.dropna(subset=['المزاج', 'الفئة_العمرية'])
        
        if not df_anova.empty and df_anova['الفئة_العمرية'].nunique() > 1:
            try:
                df_anova['الفئة_العمرية'] = df_anova['الفئة_العمرية'].astype('category')
                
                model_anova = ols('المزاج ~ C(الفئة_العمرية)', data=df_anova).fit()
                anova_table = sm.stats.anova_lm(model_anova, typ=2)
                print(get_arabic_text("\nجدول ANOVA (المزاج حسب الفئة_العمرية):"))
                display(anova_table)
            except Exception as e:
                print(get_arabic_text(f"حدث خطأ أثناء إجراء تحليل التباين (ANOVA): {str(e)}"))
        else:
            print(get_arabic_text("لا توجد بيانات كافية (أو فئة عمرية واحدة فقط) بعد إزالة القيم المفقودة لإجراء تحليل التباين (ANOVA)."))
    else:
        print(get_arabic_text("الأعمدة 'المزاج' أو 'الفئة_العمرية' غير موجودة لإجراء تحليل التباين (ANOVA)."))


    # 3. الانحدار الخطي
    required_cols_regression = ['ساعات_النوم', 'جودة_النوم', 'المزاج']
    if all(col in df.columns for col in required_cols_regression):
        df_regression = df.dropna(subset=required_cols_regression)
        
        if not df_regression.empty:
            try:
                X = df_regression[['ساعات_النوم', 'جودة_النوم']]
                y = df_regression['المزاج']
                X = sm.add_constant(X)
                
                model_ols = sm.OLS(y, X).fit()
                print(get_arabic_text("\nملخص نموذج الانحدار الخطي (المزاج ~ ساعات_النوم + جودة_النوم):"))
                display(model_ols.summary())

            except Exception as e:
                print(get_arabic_text(f"حدث خطأ أثناء بناء نموذج الانحدار الخطي: {str(e)}"))
        else:
            print(get_arabic_text("لا توجد بيانات كافية بعد إزالة القيم المفقودة لبناء نموذج الانحدار الخطي."))
    else:
        missing_cols = [col for col in required_cols_regression if col not in df.columns]
        print(get_arabic_text(f"الأعمدة المطلوبة للانحدار الخطي غير موجودة: {', '.join(missing_cols)}"))
else:
    print(get_arabic_text("لم يتم تحميل DataFrame بنجاح أو أنه فارغ. لن يتم إجراء التحليلات الإحصائية."))

!ﻲﺋﺎﺼﺣﻹﺍ ﻞﻴﻠﺤﺘﻟﺍ ﺽﺮﻐﻟ ﺡﺎﺠﻨﺑ ﺕﺎﻧﺎﻴﺒﻟﺍ ﻞﻴﻤﺤﺗ ﻢﺗ

*** (ﺔﺠﻟﺎﻌﻣ ﻱﺃ ﻞﺒﻗ) ﻞﻴﻤﺤﺘﻟﺍ ﺪﻌﺑ DataFrame ﻲﻓ ﺔﺣﺎﺘﻤﻟﺍ ﺓﺪﻤﻋﻷﺍ ***
['Id', 'وقت البدء', 'وقت الإكمال', 'الجنس :', 'كم عمرك؟', 'هل تعاني من مشاكل صحية أو نفسية تؤثر على نومك؟', 'الحالة الحالية', 'ساعات_النوم', 'متى نمت تقريبا؟', 'هل كان نومك:', 'هل تستخدم الجوال قبل النوم؟', 'هل تأخذ قيلولة خلال اليوم؟', 'ما سبب قلة نومك (لو كنت تنام أقل من 6 ساعات غالبًا)؟', 'مزاج', 'طاقة', 'تركيز', 'تسويف', 'صعوبة_اجتماعية', 'تأثير_النوم_على_النفسية', 'صف شعورك لما تنام نومًا عميقًا وكافيًا:', 'متى آخر مرة حسيت أن قلة نومك أثرت عليك فعليًا؟ وش صار؟', 'ساعات_النوم_رقمي', 'تسويف_رقمي']
['ﻞﺟﺭ' ,'ﺓﺃﺮﻣﺍ'] :': ﺲﻨﺠﻟﺍ' ﺩﻮﻤﻋ ﻲﻓ ﺔﻴﻠﺻﻷﺍ ﺓﺪﻳﺮﻔﻟﺍ ﻢﻴﻘﻟﺍ
['xa0', '18 - 25\xa0', '>30', '26 - 30\18 >'] :'؟ﻙﺮﻤﻋ ﻢﻛ' ﺩﻮﻤﻋ ﻲﻓ ﺔﻴﻠﺻﻷﺍ ﺓﺪﻳﺮﻔﻟﺍ ﻢﻴﻘﻟﺍ
['ﻊﻄﻘﺘﻣ' ,'ﻞﺻﺍﻮﺘﻣ'] :':ﻚﻣﻮﻧ ﻥﺎﻛ ﻞﻫ' ﺩﻮﻤﻋ ﻲﻓ ﺔﻴﻠﺻﻷﺍ ﺓﺪﻳﺮﻔﻟﺍ ﻢﻴﻘﻟﺍ
[2 ,1 ,3 ,5 ,4] :'ﺝﺍﺰﻣ' ﺩﻮﻤﻋ ﻲﻓ ﺔﻴﻠﺻﻷﺍ ﺓﺪﻳﺮﻔﻟﺍ ﻢﻴﻘﻟﺍ
[7.0 ,3.0 ,9.0] :'ﻲﻤﻗﺭ_ﻡﻮﻨﻟﺍ_ﺕﺎﻋﺎﺳ' ﺩﻮﻤﻋ ﻲﻓ ﺔﻴﻠﺻﻷﺍ ﺓﺪﻳﺮﻔﻟﺍ ﻢﻴﻘﻟﺍ
------------------------------------------------------


C:\Users\gknow\AppData\Local\Temp\ipykernel_29552\567445678.py:87: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ساعات_النوم'].fillna(df['ساعات_النوم'].mean(), inplace=True)
C:\Users\gknow\AppData\Local\Temp\ipykernel_29552\567445678.py:133: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

,sum_sq,df,F,PR(>F)
C(الفئة_العمرية),1793.370130,5.0,249.592533,2.229749e-34
Residual,110.651948,77.0,NaN,NaN



:(ﻡﻮﻨﻟﺍ_ﺓﺩﻮﺟ + ﻡﻮﻨﻟﺍ_ﺕﺎﻋﺎﺳ ~ ﺝﺍﺰﻤﻟﺍ) ﻲﻄﺨﻟﺍ ﺭﺍﺪﺤﻧﻻﺍ ﺝﺫﻮﻤﻧ ﺺﺨﻠﻣ


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 المزاج   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     2.980
Date:                Mon, 21 Jul 2025   Prob (F-statistic):             0.0568
Time:                        19:34:53   Log-Likelihood:                -122.54
No. Observations:                  79   AIC:                             251.1
Df Residuals:                      76   BIC:                             258.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           1.7308      0.543      3.190      0.002       0.650       2.811
ساعات_النوم     0.1818      0.081      2.253      0.027       0.021       0.342
جودة_النوم      0.0039      0.073      0.053      0.958      -0.142       0.150
==============================================================================
Omnibus:                        2.602   Durbin-Watson:                   1.970
Prob(Omnibus):                  0.272   Jarque-Bera (JB):                2.399
Skew:                          -0.341   Prob(JB):                        0.301
Kurtosis:                       2.487   Cond. No.                         32.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
import pandas as pd
from scipy import stats

# تنظيف أسماء الأعمدة من الفراغات
df.columns = df.columns.str.strip()

# التأكد من عمود ساعات النوم الرقمي
if 'ساعات_النوم_رقمي' not in df.columns:
    if pd.api.types.is_numeric_dtype(df['ساعات_النوم']):
        df['ساعات_النوم_رقمي'] = df['ساعات_النوم'].astype(float)
    else:
        df['ساعات_النوم_رقمي'] = df['ساعات_النوم'].astype(str).str.extract('(\d+)').astype(float)

# تحويل عمود المزاج ('المزاج') لرقمي
df['المزاج'] = pd.to_numeric(df['المزاج'], errors='coerce')

# تحليل الموظفين
employees = df[df['الحالة الحالية'].str.contains('موظف', na=False)]
less_than_6 = employees[employees['ساعات_النوم_رقمي'] < 6]
percentage = len(less_than_6) / len(employees) * 100
print(f"الموظفون الذين ينامون أقل من 6 ساعات: {percentage:.0f}%")

# تأثير استخدام الجوال قبل النوم على المزاج
phone_users = df[df['هل تستخدم الجوال قبل النوم؟'] != 'لا']
no_phone = df[df['هل تستخدم الجوال قبل النوم؟'] == 'لا']

phone_users_mood = phone_users['المزاج'].dropna()
no_phone_mood = no_phone['المزاج'].dropna()

t_stat, p_value = stats.ttest_ind(phone_users_mood, no_phone_mood)
diff = no_phone_mood.mean() - phone_users_mood.mean()

print(f"الفرق في المزاج بين مستخدمي الجوال وغيرهم: {diff:.1f} نقطة")
print(f"قيمة p لاختبار t: {p_value:.4f}")


الموظفون الذين ينامون أقل من 6 ساعات: 21%
الفرق في المزاج بين مستخدمي الجوال وغيرهم: nan نقطة
قيمة p لاختبار t: nan


C:\Users\gknow\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\_lib\deprecation.py:234: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)


In [14]:
print("عدد مستخدمي الجوال مع بيانات المزاج:", phone_users_mood.count())
print("عدد غير مستخدمي الجوال مع بيانات المزاج:", no_phone_mood.count())


عدد مستخدمي الجوال مع بيانات المزاج: 79
عدد غير مستخدمي الجوال مع بيانات المزاج: 0
